# Read the Test Data

In [28]:
import pandas as pd
import numpy as np

test_df = pd.read_csv('/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/cleaned_data/test.cleaned_readme_data.csv')
test_df.head()

,Unnamed: 0,id,link,summary,readme
0,0,136617793,https://api.github.com/repos/AllenWangle/Resea...,"A paper list of my research line, welcome to d...",awesome paper list prs welcomereadmeassetsprs...
1,1,136860222,https://api.github.com/repos/heikipikker/firmw...,Toolkit to emulate firmware and analyse it for...,firmware analysis toolkit fat is a toolkit bu...
2,2,136792496,https://api.github.com/repos/shaunagm/governan...,Resources to help you choose a governance syst...,governance resources welcome to governance re...
3,3,136898852,https://api.github.com/repos/srmagura/expo-pixi,Tools for using pixi.js in Expo,expopixi tools to use pixijshttpwwwpixijscom ...
4,4,136699628,https://api.github.com/repos/tenfinney/graphql...,A library to help construct a graphql-js serve...,relay library for graphqljs this is a library...


In [29]:
test_df.drop('Unnamed: 0', 1)

,id,link,summary,readme
0,136617793,https://api.github.com/repos/AllenWangle/Resea...,"A paper list of my research line, welcome to d...",awesome paper list prs welcomereadmeassetsprs...
1,136860222,https://api.github.com/repos/heikipikker/firmw...,Toolkit to emulate firmware and analyse it for...,firmware analysis toolkit fat is a toolkit bu...
2,136792496,https://api.github.com/repos/shaunagm/governan...,Resources to help you choose a governance syst...,governance resources welcome to governance re...
3,136898852,https://api.github.com/repos/srmagura/expo-pixi,Tools for using pixi.js in Expo,expopixi tools to use pixijshttpwwwpixijscom ...
4,136699628,https://api.github.com/repos/tenfinney/graphql...,A library to help construct a graphql-js serve...,relay library for graphqljs this is a library...
...,...,...,...,...
1087,136572890,https://api.github.com/repos/jigjunior/Tooltip,Simple to use customizable Android Tooltips li...,android tooltips license mithttpsimgshieldsio...
1088,137431357,https://api.github.com/repos/suresh-H5/svg.dra...,An extension for the svg.js library to make el...,svgdraggablejs a plugin for the svgdotjsgithu...
1089,137472649,https://api.github.com/repos/Kartstig/BAC0,BAC0 - Library depending on BACpypes (Python 3...,bac0 buildstatus coverage docs image httpsbadg...
1090,137472684,https://api.github.com/repos/ericpoirier/opena...,OpenADx is about an ecosystem. Each company/pa...,openadxeclipseorg the openadxeclipseorghttpso...


# Lexrank

In [3]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lex_rank import LexRankSummarizer



LANGUAGE = 'english'
SENTENCES_COUNT = 1

def get_lex_rank_summarization(source):
    parser = PlaintextParser.from_string(source, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    Summarizer = LexRankSummarizer
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    results = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        results.append(str(sentence))
    return results[0]

# Edmunson

In [4]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.edmundson import EdmundsonSummarizer



LANGUAGE = 'english'
SENTENCES_COUNT = 1

def get_edmunson_summarization(source):
    parser = PlaintextParser.from_string(source, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = EdmundsonSummarizer(stemmer)
    summarizer.bonus_words = parser.significant_words
    summarizer.stigma_words = parser.stigma_words
    summarizer.null_words = get_stop_words(LANGUAGE)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    results = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        results.append(str(sentence))
    return results[0]

# Apply the Extractive Summarization Method

In [5]:
# Run the extractive summarization on the Readme content in the dataset 
readme_lexrank = test_df['readme'].apply(get_lex_rank_summarization)

In [6]:
readme_edmunson = test_df['readme'].apply(get_edmunson_summarization)

In [7]:
readme_edmunson

0       awesome paper list prs welcomereadmeassetsprsw...
1       firmware analysis toolkit fat is a toolkit bui...
2       governance resources welcome to governance res...
3       expopixi tools to use pixijshttpwwwpixijscom i...
4       relay library for graphqljs this is a library ...
                              ...                        
1087    android tooltips license mithttpsimgshieldsiob...
1088    svgdraggablejs a plugin for the svgdotjsgithub...
1089    bac0 buildstatus coverage docs image httpsbadg...
1090    openadxeclipseorg the openadxeclipseorghttpsop...
1091    xiaomi mi flora plant sensor mqtt clientdaemon...
Name: readme, Length: 1092, dtype: object

In [8]:
test_df_copy = test_df.copy()

# Calculate Rouge Score by Extractive Summarization

In [43]:
from pyrouge import Rouge155
def calculate_rouge_score(model_index, system_dir, model_dir):
    r = Rouge155()
    r.system_dir = system_dir
    r.model_dir = model_dir
    r.system_filename_pattern = 'system.(\d+).txt'
    r.model_filename_pattern = f'model.{model_index}.#ID#.txt'

    output = r.convert_and_evaluate()
    output_dict = r.output_to_dict(output)
    return output, output_dict

SYS_ROOT_DIR = '/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/system_summaries'
MOD_ROOT_DIR = '/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/model_summaries'
MOD_ROOT_DIR02 = '/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/model_summaries_edmunson'
MOD_ROOT_DIR03 = '/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/model_summaries_leadcm'



In [44]:
lex_rank_rouge = calculate_rouge_score('A', SYS_ROOT_DIR, MOD_ROOT_DIR)

2020-06-19 22:29:11,102 [MainThread  ] [INFO ]  Writing summaries.
2020-06-19 22:29:11,103 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmp5jemm0ji/system and model files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmp5jemm0ji/model.
2020-06-19 22:29:11,103 [MainThread  ] [INFO ]  Processing files in /Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/system_summaries.
2020-06-19 22:29:11,105 [MainThread  ] [INFO ]  Processing system.80.txt.
2020-06-19 22:29:11,106 [MainThread  ] [INFO ]  Processing system.1064.txt.
2020-06-19 22:29:11,107 [MainThread  ] [INFO ]  Processing system.94.txt.
2020-06-19 22:29:11,107 [MainThread  ] [INFO ]  Processing system.1070.txt.
2020-06-19 22:29:11,109 [MainThread  ] [INFO ]  Processing system.1058.txt.
2020-06-19 22:29:11,110 [MainThread  ] [INFO ]  Processing system.364.txt.
2020-06-19 22:29:11,110 [MainThread  ] [INFO ]  Processing system.43.txt.


2020-06-19 22:29:11,199 [MainThread  ] [INFO ]  Processing system.206.txt.
2020-06-19 22:29:11,200 [MainThread  ] [INFO ]  Processing system.560.txt.
2020-06-19 22:29:11,201 [MainThread  ] [INFO ]  Processing system.992.txt.
2020-06-19 22:29:11,201 [MainThread  ] [INFO ]  Processing system.986.txt.
2020-06-19 22:29:11,202 [MainThread  ] [INFO ]  Processing system.776.txt.
2020-06-19 22:29:11,203 [MainThread  ] [INFO ]  Processing system.762.txt.
2020-06-19 22:29:11,204 [MainThread  ] [INFO ]  Processing system.789.txt.
2020-06-19 22:29:11,205 [MainThread  ] [INFO ]  Processing system.951.txt.
2020-06-19 22:29:11,206 [MainThread  ] [INFO ]  Processing system.945.txt.
2020-06-19 22:29:11,207 [MainThread  ] [INFO ]  Processing system.979.txt.
2020-06-19 22:29:11,208 [MainThread  ] [INFO ]  Processing system.158.txt.
2020-06-19 22:29:11,209 [MainThread  ] [INFO ]  Processing system.164.txt.
2020-06-19 22:29:11,210 [MainThread  ] [INFO ]  Processing system.602.txt.
2020-06-19 22:29:11,211 [

2020-06-19 22:29:11,307 [MainThread  ] [INFO ]  Processing system.375.txt.
2020-06-19 22:29:11,308 [MainThread  ] [INFO ]  Processing system.413.txt.
2020-06-19 22:29:11,309 [MainThread  ] [INFO ]  Processing system.52.txt.
2020-06-19 22:29:11,310 [MainThread  ] [INFO ]  Processing system.822.txt.
2020-06-19 22:29:11,310 [MainThread  ] [INFO ]  Processing system.836.txt.
2020-06-19 22:29:11,311 [MainThread  ] [INFO ]  Processing system.188.txt.
2020-06-19 22:29:11,312 [MainThread  ] [INFO ]  Processing system.639.txt.
2020-06-19 22:29:11,313 [MainThread  ] [INFO ]  Processing system.163.txt.
2020-06-19 22:29:11,314 [MainThread  ] [INFO ]  Processing system.605.txt.
2020-06-19 22:29:11,315 [MainThread  ] [INFO ]  Processing system.611.txt.
2020-06-19 22:29:11,315 [MainThread  ] [INFO ]  Processing system.177.txt.
2020-06-19 22:29:11,316 [MainThread  ] [INFO ]  Processing system.956.txt.
2020-06-19 22:29:11,317 [MainThread  ] [INFO ]  Processing system.942.txt.
2020-06-19 22:29:11,318 [M

2020-06-19 22:29:11,404 [MainThread  ] [INFO ]  Processing system.879.txt.
2020-06-19 22:29:11,405 [MainThread  ] [INFO ]  Processing system.9.txt.
2020-06-19 22:29:11,405 [MainThread  ] [INFO ]  Processing system.448.txt.
2020-06-19 22:29:11,406 [MainThread  ] [INFO ]  Processing system.312.txt.
2020-06-19 22:29:11,407 [MainThread  ] [INFO ]  Processing system.474.txt.
2020-06-19 22:29:11,408 [MainThread  ] [INFO ]  Processing system.35.txt.
2020-06-19 22:29:11,409 [MainThread  ] [INFO ]  Processing system.21.txt.
2020-06-19 22:29:11,409 [MainThread  ] [INFO ]  Processing system.460.txt.
2020-06-19 22:29:11,410 [MainThread  ] [INFO ]  Processing system.306.txt.
2020-06-19 22:29:11,411 [MainThread  ] [INFO ]  Processing system.1012.txt.
2020-06-19 22:29:11,412 [MainThread  ] [INFO ]  Processing system.1006.txt.
2020-06-19 22:29:11,412 [MainThread  ] [INFO ]  Processing system.1010.txt.
2020-06-19 22:29:11,413 [MainThread  ] [INFO ]  Processing system.1004.txt.
2020-06-19 22:29:11,414 [

2020-06-19 22:29:11,498 [MainThread  ] [INFO ]  Processing system.923.txt.
2020-06-19 22:29:11,499 [MainThread  ] [INFO ]  Processing system.102.txt.
2020-06-19 22:29:11,500 [MainThread  ] [INFO ]  Processing system.664.txt.
2020-06-19 22:29:11,501 [MainThread  ] [INFO ]  Processing system.670.txt.
2020-06-19 22:29:11,501 [MainThread  ] [INFO ]  Processing system.116.txt.
2020-06-19 22:29:11,502 [MainThread  ] [INFO ]  Processing system.658.txt.
2020-06-19 22:29:11,503 [MainThread  ] [INFO ]  Processing system.880.txt.
2020-06-19 22:29:11,504 [MainThread  ] [INFO ]  Processing system.894.txt.
2020-06-19 22:29:11,505 [MainThread  ] [INFO ]  Processing system.843.txt.
2020-06-19 22:29:11,506 [MainThread  ] [INFO ]  Processing system.857.txt.
2020-06-19 22:29:11,507 [MainThread  ] [INFO ]  Processing system.27.txt.
2020-06-19 22:29:11,507 [MainThread  ] [INFO ]  Processing system.466.txt.
2020-06-19 22:29:11,508 [MainThread  ] [INFO ]  Processing system.300.txt.
2020-06-19 22:29:11,509 [M

...

2020-06-19 22:29:12,842 [MainThread  ] [INFO ]  Processing model.A.490.txt.
2020-06-19 22:29:12,843 [MainThread  ] [INFO ]  Processing model.A.484.txt.
2020-06-19 22:29:12,844 [MainThread  ] [INFO ]  Processing model.A.269.txt.
2020-06-19 22:29:12,845 [MainThread  ] [INFO ]  Processing model.A.241.txt.
2020-06-19 22:29:12,846 [MainThread  ] [INFO ]  Processing model.A.527.txt.
2020-06-19 22:29:12,847 [MainThread  ] [INFO ]  Processing model.A.533.txt.
2020-06-19 22:29:12,848 [MainThread  ] [INFO ]  Processing model.A.255.txt.
2020-06-19 22:29:12,848 [MainThread  ] [INFO ]  Processing model.A.11.txt.
2020-06-19 22:29:12,849 [MainThread  ] [INFO ]  Processing model.A.282.txt.
2020-06-19 22:29:12,850 [MainThread  ] [INFO ]  Processing model.A.296.txt.
2020-06-19 22:29:12,851 [MainThread  ] [INFO ]  Processing model.A.39.txt.
2020-06-19 22:29:12,852 [MainThread  ] [INFO ]  Processing model.A.719.txt.
2020-06-19 22:29:12,853 [MainThread  ] [INFO ]  Processing model.A.725.txt.
2020-06-19 22:

2020-06-19 22:29:12,946 [MainThread  ] [INFO ]  Processing model.A.871.txt.
2020-06-19 22:29:12,948 [MainThread  ] [INFO ]  Processing model.A.865.txt.
2020-06-19 22:29:12,949 [MainThread  ] [INFO ]  Processing model.A.695.txt.
2020-06-19 22:29:12,949 [MainThread  ] [INFO ]  Processing model.A.681.txt.
2020-06-19 22:29:12,950 [MainThread  ] [INFO ]  Processing model.A.859.txt.
2020-06-19 22:29:12,951 [MainThread  ] [INFO ]  Processing model.A.118.txt.
2020-06-19 22:29:12,953 [MainThread  ] [INFO ]  Processing model.A.656.txt.
2020-06-19 22:29:12,954 [MainThread  ] [INFO ]  Processing model.A.130.txt.
2020-06-19 22:29:12,955 [MainThread  ] [INFO ]  Processing model.A.124.txt.
2020-06-19 22:29:12,956 [MainThread  ] [INFO ]  Processing model.A.642.txt.
2020-06-19 22:29:12,956 [MainThread  ] [INFO ]  Processing model.A.125.txt.
2020-06-19 22:29:12,957 [MainThread  ] [INFO ]  Processing model.A.643.txt.
2020-06-19 22:29:12,958 [MainThread  ] [INFO ]  Processing model.A.657.txt.
2020-06-19 2

2020-06-19 22:29:13,058 [MainThread  ] [INFO ]  Processing model.A.779.txt.
2020-06-19 22:29:13,059 [MainThread  ] [INFO ]  Processing model.A.989.txt.
2020-06-19 22:29:13,060 [MainThread  ] [INFO ]  Processing model.A.751.txt.
2020-06-19 22:29:13,061 [MainThread  ] [INFO ]  Processing model.A.745.txt.
2020-06-19 22:29:13,062 [MainThread  ] [INFO ]  Processing model.A.976.txt.
2020-06-19 22:29:13,063 [MainThread  ] [INFO ]  Processing model.A.962.txt.
2020-06-19 22:29:13,064 [MainThread  ] [INFO ]  Processing model.A.792.txt.
2020-06-19 22:29:13,064 [MainThread  ] [INFO ]  Processing model.A.786.txt.
2020-06-19 22:29:13,065 [MainThread  ] [INFO ]  Processing model.A.948.txt.
2020-06-19 22:29:13,066 [MainThread  ] [INFO ]  Processing model.A.790.txt.
2020-06-19 22:29:13,067 [MainThread  ] [INFO ]  Processing model.A.784.txt.
2020-06-19 22:29:13,068 [MainThread  ] [INFO ]  Processing model.A.974.txt.
2020-06-19 22:29:13,069 [MainThread  ] [INFO ]  Processing model.A.960.txt.
2020-06-19 2

2020-06-19 22:29:13,153 [MainThread  ] [INFO ]  Processing model.A.187.txt.
2020-06-19 22:29:13,154 [MainThread  ] [INFO ]  Processing model.A.839.txt.
2020-06-19 22:29:13,155 [MainThread  ] [INFO ]  Processing model.A.193.txt.
2020-06-19 22:29:13,155 [MainThread  ] [INFO ]  Processing model.A.178.txt.
2020-06-19 22:29:13,156 [MainThread  ] [INFO ]  Processing model.A.144.txt.
2020-06-19 22:29:13,157 [MainThread  ] [INFO ]  Processing model.A.622.txt.
2020-06-19 22:29:13,158 [MainThread  ] [INFO ]  Processing model.A.636.txt.
2020-06-19 22:29:13,159 [MainThread  ] [INFO ]  Processing model.A.150.txt.
2020-06-19 22:29:13,159 [MainThread  ] [INFO ]  Processing model.A.637.txt.
2020-06-19 22:29:13,160 [MainThread  ] [INFO ]  Processing model.A.151.txt.
2020-06-19 22:29:13,161 [MainThread  ] [INFO ]  Processing model.A.145.txt.
2020-06-19 22:29:13,162 [MainThread  ] [INFO ]  Processing model.A.623.txt.
2020-06-19 22:29:13,162 [MainThread  ] [INFO ]  Processing model.A.179.txt.
2020-06-19 2

2020-06-19 22:29:13,245 [MainThread  ] [INFO ]  Processing model.A.60.txt.
2020-06-19 22:29:13,246 [MainThread  ] [INFO ]  Processing model.A.74.txt.
2020-06-19 22:29:13,247 [MainThread  ] [INFO ]  Processing model.A.754.txt.
2020-06-19 22:29:13,247 [MainThread  ] [INFO ]  Processing model.A.998.txt.
2020-06-19 22:29:13,248 [MainThread  ] [INFO ]  Processing model.A.740.txt.
2020-06-19 22:29:13,249 [MainThread  ] [INFO ]  Processing model.A.768.txt.
2020-06-19 22:29:13,250 [MainThread  ] [INFO ]  Processing model.A.797.txt.
2020-06-19 22:29:13,251 [MainThread  ] [INFO ]  Processing model.A.783.txt.
2020-06-19 22:29:13,252 [MainThread  ] [INFO ]  Processing model.A.973.txt.
2020-06-19 22:29:13,253 [MainThread  ] [INFO ]  Processing model.A.967.txt.
2020-06-19 22:29:13,255 [MainThread  ] [INFO ]  Saved processed files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmp5jemm0ji/model.
2020-06-19 22:29:14,510 [MainThread  ] [INFO ]  Written ROUGE configuration to /var/folders/0c/zfyk8f

In [45]:
edmunson_rouge = calculate_rouge_score('B', SYS_ROOT_DIR, MOD_ROOT_DIR02)

2020-06-19 22:29:37,038 [MainThread  ] [INFO ]  Writing summaries.
2020-06-19 22:29:37,040 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpp5ujkn78/system and model files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpp5ujkn78/model.
2020-06-19 22:29:37,041 [MainThread  ] [INFO ]  Processing files in /Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/system_summaries.
2020-06-19 22:29:37,050 [MainThread  ] [INFO ]  Processing system.80.txt.
2020-06-19 22:29:37,079 [MainThread  ] [INFO ]  Processing system.1064.txt.
2020-06-19 22:29:37,088 [MainThread  ] [INFO ]  Processing system.94.txt.
2020-06-19 22:29:37,089 [MainThread  ] [INFO ]  Processing system.1070.txt.
2020-06-19 22:29:37,090 [MainThread  ] [INFO ]  Processing system.1058.txt.
2020-06-19 22:29:37,099 [MainThread  ] [INFO ]  Processing system.364.txt.
2020-06-19 22:29:37,102 [MainThread  ] [INFO ]  Processing system.43.txt.


2020-06-19 22:29:37,388 [MainThread  ] [INFO ]  Processing system.206.txt.
2020-06-19 22:29:37,389 [MainThread  ] [INFO ]  Processing system.560.txt.
2020-06-19 22:29:37,391 [MainThread  ] [INFO ]  Processing system.992.txt.
2020-06-19 22:29:37,392 [MainThread  ] [INFO ]  Processing system.986.txt.
2020-06-19 22:29:37,393 [MainThread  ] [INFO ]  Processing system.776.txt.
2020-06-19 22:29:37,394 [MainThread  ] [INFO ]  Processing system.762.txt.
2020-06-19 22:29:37,394 [MainThread  ] [INFO ]  Processing system.789.txt.
2020-06-19 22:29:37,395 [MainThread  ] [INFO ]  Processing system.951.txt.
2020-06-19 22:29:37,396 [MainThread  ] [INFO ]  Processing system.945.txt.
2020-06-19 22:29:37,397 [MainThread  ] [INFO ]  Processing system.979.txt.
2020-06-19 22:29:37,398 [MainThread  ] [INFO ]  Processing system.158.txt.
2020-06-19 22:29:37,399 [MainThread  ] [INFO ]  Processing system.164.txt.
2020-06-19 22:29:37,401 [MainThread  ] [INFO ]  Processing system.602.txt.
2020-06-19 22:29:37,402 [

2020-06-19 22:29:37,492 [MainThread  ] [INFO ]  Processing system.375.txt.
2020-06-19 22:29:37,492 [MainThread  ] [INFO ]  Processing system.413.txt.
2020-06-19 22:29:37,493 [MainThread  ] [INFO ]  Processing system.52.txt.
2020-06-19 22:29:37,494 [MainThread  ] [INFO ]  Processing system.822.txt.
2020-06-19 22:29:37,495 [MainThread  ] [INFO ]  Processing system.836.txt.
2020-06-19 22:29:37,496 [MainThread  ] [INFO ]  Processing system.188.txt.
2020-06-19 22:29:37,496 [MainThread  ] [INFO ]  Processing system.639.txt.
2020-06-19 22:29:37,497 [MainThread  ] [INFO ]  Processing system.163.txt.
2020-06-19 22:29:37,498 [MainThread  ] [INFO ]  Processing system.605.txt.
2020-06-19 22:29:37,499 [MainThread  ] [INFO ]  Processing system.611.txt.
2020-06-19 22:29:37,500 [MainThread  ] [INFO ]  Processing system.177.txt.
2020-06-19 22:29:37,501 [MainThread  ] [INFO ]  Processing system.956.txt.
2020-06-19 22:29:37,502 [MainThread  ] [INFO ]  Processing system.942.txt.
2020-06-19 22:29:37,503 [M

2020-06-19 22:29:37,585 [MainThread  ] [INFO ]  Processing system.879.txt.
2020-06-19 22:29:37,586 [MainThread  ] [INFO ]  Processing system.9.txt.
2020-06-19 22:29:37,587 [MainThread  ] [INFO ]  Processing system.448.txt.
2020-06-19 22:29:37,588 [MainThread  ] [INFO ]  Processing system.312.txt.
2020-06-19 22:29:37,588 [MainThread  ] [INFO ]  Processing system.474.txt.
2020-06-19 22:29:37,589 [MainThread  ] [INFO ]  Processing system.35.txt.
2020-06-19 22:29:37,590 [MainThread  ] [INFO ]  Processing system.21.txt.
2020-06-19 22:29:37,591 [MainThread  ] [INFO ]  Processing system.460.txt.
2020-06-19 22:29:37,592 [MainThread  ] [INFO ]  Processing system.306.txt.
2020-06-19 22:29:37,592 [MainThread  ] [INFO ]  Processing system.1012.txt.
2020-06-19 22:29:37,593 [MainThread  ] [INFO ]  Processing system.1006.txt.
2020-06-19 22:29:37,594 [MainThread  ] [INFO ]  Processing system.1010.txt.
2020-06-19 22:29:37,595 [MainThread  ] [INFO ]  Processing system.1004.txt.
2020-06-19 22:29:37,596 [

2020-06-19 22:29:37,679 [MainThread  ] [INFO ]  Processing system.923.txt.
2020-06-19 22:29:37,680 [MainThread  ] [INFO ]  Processing system.102.txt.
2020-06-19 22:29:37,681 [MainThread  ] [INFO ]  Processing system.664.txt.
2020-06-19 22:29:37,682 [MainThread  ] [INFO ]  Processing system.670.txt.
2020-06-19 22:29:37,683 [MainThread  ] [INFO ]  Processing system.116.txt.
2020-06-19 22:29:37,683 [MainThread  ] [INFO ]  Processing system.658.txt.
2020-06-19 22:29:37,684 [MainThread  ] [INFO ]  Processing system.880.txt.
2020-06-19 22:29:37,685 [MainThread  ] [INFO ]  Processing system.894.txt.
2020-06-19 22:29:37,686 [MainThread  ] [INFO ]  Processing system.843.txt.
2020-06-19 22:29:37,687 [MainThread  ] [INFO ]  Processing system.857.txt.
2020-06-19 22:29:37,688 [MainThread  ] [INFO ]  Processing system.27.txt.
2020-06-19 22:29:37,689 [MainThread  ] [INFO ]  Processing system.466.txt.
2020-06-19 22:29:37,690 [MainThread  ] [INFO ]  Processing system.300.txt.
2020-06-19 22:29:37,691 [M

...

2020-06-19 22:29:39,122 [MainThread  ] [INFO ]  Processing model.B.194.txt.
2020-06-19 22:29:39,123 [MainThread  ] [INFO ]  Processing model.B.97.txt.
2020-06-19 22:29:39,124 [MainThread  ] [INFO ]  Processing model.B.619.txt.
2020-06-19 22:29:39,125 [MainThread  ] [INFO ]  Processing model.B.68.txt.
2020-06-19 22:29:39,126 [MainThread  ] [INFO ]  Processing model.B.40.txt.
2020-06-19 22:29:39,127 [MainThread  ] [INFO ]  Processing model.B.625.txt.
2020-06-19 22:29:39,128 [MainThread  ] [INFO ]  Processing model.B.143.txt.
2020-06-19 22:29:39,128 [MainThread  ] [INFO ]  Processing model.B.157.txt.
2020-06-19 22:29:39,129 [MainThread  ] [INFO ]  Processing model.B.631.txt.
2020-06-19 22:29:39,130 [MainThread  ] [INFO ]  Processing model.B.54.txt.
2020-06-19 22:29:39,131 [MainThread  ] [INFO ]  Processing model.B.609.txt.
2020-06-19 22:29:39,132 [MainThread  ] [INFO ]  Processing model.B.78.txt.
2020-06-19 22:29:39,133 [MainThread  ] [INFO ]  Processing model.B.50.txt.
2020-06-19 22:29:3

2020-06-19 22:29:39,218 [MainThread  ] [INFO ]  Processing model.B.568.txt.
2020-06-19 22:29:39,219 [MainThread  ] [INFO ]  Processing model.B.1020.txt.
2020-06-19 22:29:39,220 [MainThread  ] [INFO ]  Processing model.B.232.txt.
2020-06-19 22:29:39,221 [MainThread  ] [INFO ]  Processing model.B.1008.txt.
2020-06-19 22:29:39,222 [MainThread  ] [INFO ]  Processing model.B.554.txt.
2020-06-19 22:29:39,223 [MainThread  ] [INFO ]  Processing model.B.540.txt.
2020-06-19 22:29:39,224 [MainThread  ] [INFO ]  Processing model.B.226.txt.
2020-06-19 22:29:39,224 [MainThread  ] [INFO ]  Processing model.B.597.txt.
2020-06-19 22:29:39,225 [MainThread  ] [INFO ]  Processing model.B.583.txt.
2020-06-19 22:29:39,226 [MainThread  ] [INFO ]  Processing model.B.756.txt.
2020-06-19 22:29:39,227 [MainThread  ] [INFO ]  Processing model.B.742.txt.
2020-06-19 22:29:39,228 [MainThread  ] [INFO ]  Processing model.B.971.txt.
2020-06-19 22:29:39,229 [MainThread  ] [INFO ]  Processing model.B.965.txt.
2020-06-19

2020-06-19 22:29:39,339 [MainThread  ] [INFO ]  Processing model.B.321.txt.
2020-06-19 22:29:39,340 [MainThread  ] [INFO ]  Processing model.B.447.txt.
2020-06-19 22:29:39,341 [MainThread  ] [INFO ]  Processing model.B.876.txt.
2020-06-19 22:29:39,341 [MainThread  ] [INFO ]  Processing model.B.862.txt.
2020-06-19 22:29:39,342 [MainThread  ] [INFO ]  Processing model.B.692.txt.
2020-06-19 22:29:39,343 [MainThread  ] [INFO ]  Processing model.B.686.txt.
2020-06-19 22:29:39,344 [MainThread  ] [INFO ]  Processing model.B.679.txt.
2020-06-19 22:29:39,345 [MainThread  ] [INFO ]  Processing model.B.137.txt.
2020-06-19 22:29:39,346 [MainThread  ] [INFO ]  Processing model.B.34.txt.
2020-06-19 22:29:39,346 [MainThread  ] [INFO ]  Processing model.B.651.txt.
2020-06-19 22:29:39,347 [MainThread  ] [INFO ]  Processing model.B.889.txt.
2020-06-19 22:29:39,348 [MainThread  ] [INFO ]  Processing model.B.645.txt.
2020-06-19 22:29:39,349 [MainThread  ] [INFO ]  Processing model.B.20.txt.
2020-06-19 22:

2020-06-19 22:29:39,434 [MainThread  ] [INFO ]  Processing model.B.246.txt.
2020-06-19 22:29:39,435 [MainThread  ] [INFO ]  Processing model.B.252.txt.
2020-06-19 22:29:39,436 [MainThread  ] [INFO ]  Processing model.B.1068.txt.
2020-06-19 22:29:39,437 [MainThread  ] [INFO ]  Processing model.B.534.txt.
2020-06-19 22:29:39,439 [MainThread  ] [INFO ]  Processing model.B.1083.txt.
2020-06-19 22:29:39,440 [MainThread  ] [INFO ]  Processing model.B.285.txt.
2020-06-19 22:29:39,441 [MainThread  ] [INFO ]  Processing model.B.291.txt.
2020-06-19 22:29:39,442 [MainThread  ] [INFO ]  Processing model.B.722.txt.
2020-06-19 22:29:39,442 [MainThread  ] [INFO ]  Processing model.B.736.txt.
2020-06-19 22:29:39,443 [MainThread  ] [INFO ]  Processing model.B.905.txt.
2020-06-19 22:29:39,444 [MainThread  ] [INFO ]  Processing model.B.911.txt.
2020-06-19 22:29:39,445 [MainThread  ] [INFO ]  Processing model.B.939.txt.
2020-06-19 22:29:39,446 [MainThread  ] [INFO ]  Processing model.B.938.txt.
2020-06-19

2020-06-19 22:29:39,533 [MainThread  ] [INFO ]  Processing model.B.867.txt.
2020-06-19 22:29:39,534 [MainThread  ] [INFO ]  Processing model.B.31.txt.
2020-06-19 22:29:39,534 [MainThread  ] [INFO ]  Processing model.B.654.txt.
2020-06-19 22:29:39,535 [MainThread  ] [INFO ]  Processing model.B.132.txt.
2020-06-19 22:29:39,536 [MainThread  ] [INFO ]  Processing model.B.126.txt.
2020-06-19 22:29:39,537 [MainThread  ] [INFO ]  Processing model.B.640.txt.
2020-06-19 22:29:39,539 [MainThread  ] [INFO ]  Processing model.B.898.txt.
2020-06-19 22:29:39,539 [MainThread  ] [INFO ]  Processing model.B.25.txt.
2020-06-19 22:29:39,540 [MainThread  ] [INFO ]  Processing model.B.668.txt.
2020-06-19 22:29:39,541 [MainThread  ] [INFO ]  Processing model.B.19.txt.
2020-06-19 22:29:39,542 [MainThread  ] [INFO ]  Saved processed files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpp5ujkn78/model.
2020-06-19 22:29:40,805 [MainThread  ] [INFO ]  Written ROUGE configuration to /var/folders/0c/zfyk8fc

In [46]:
leadcm_rouge = calculate_rouge_score('A', SYS_ROOT_DIR, MOD_ROOT_DIR03)

2020-06-19 22:30:09,267 [MainThread  ] [INFO ]  Writing summaries.
2020-06-19 22:30:09,268 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpys1wssh7/system and model files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpys1wssh7/model.
2020-06-19 22:30:09,269 [MainThread  ] [INFO ]  Processing files in /Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/output/system_summaries.
2020-06-19 22:30:09,274 [MainThread  ] [INFO ]  Processing system.80.txt.
2020-06-19 22:30:09,277 [MainThread  ] [INFO ]  Processing system.1064.txt.
2020-06-19 22:30:09,282 [MainThread  ] [INFO ]  Processing system.94.txt.
2020-06-19 22:30:09,285 [MainThread  ] [INFO ]  Processing system.1070.txt.
2020-06-19 22:30:09,287 [MainThread  ] [INFO ]  Processing system.1058.txt.
2020-06-19 22:30:09,289 [MainThread  ] [INFO ]  Processing system.364.txt.
2020-06-19 22:30:09,291 [MainThread  ] [INFO ]  Processing system.43.txt.


2020-06-19 22:30:09,422 [MainThread  ] [INFO ]  Processing system.206.txt.
2020-06-19 22:30:09,423 [MainThread  ] [INFO ]  Processing system.560.txt.
2020-06-19 22:30:09,423 [MainThread  ] [INFO ]  Processing system.992.txt.
2020-06-19 22:30:09,424 [MainThread  ] [INFO ]  Processing system.986.txt.
2020-06-19 22:30:09,425 [MainThread  ] [INFO ]  Processing system.776.txt.
2020-06-19 22:30:09,426 [MainThread  ] [INFO ]  Processing system.762.txt.
2020-06-19 22:30:09,427 [MainThread  ] [INFO ]  Processing system.789.txt.
2020-06-19 22:30:09,428 [MainThread  ] [INFO ]  Processing system.951.txt.
2020-06-19 22:30:09,428 [MainThread  ] [INFO ]  Processing system.945.txt.
2020-06-19 22:30:09,429 [MainThread  ] [INFO ]  Processing system.979.txt.
2020-06-19 22:30:09,430 [MainThread  ] [INFO ]  Processing system.158.txt.
2020-06-19 22:30:09,431 [MainThread  ] [INFO ]  Processing system.164.txt.
2020-06-19 22:30:09,433 [MainThread  ] [INFO ]  Processing system.602.txt.
2020-06-19 22:30:09,434 [

2020-06-19 22:30:09,584 [MainThread  ] [INFO ]  Processing system.375.txt.
2020-06-19 22:30:09,585 [MainThread  ] [INFO ]  Processing system.413.txt.
2020-06-19 22:30:09,586 [MainThread  ] [INFO ]  Processing system.52.txt.
2020-06-19 22:30:09,586 [MainThread  ] [INFO ]  Processing system.822.txt.
2020-06-19 22:30:09,600 [MainThread  ] [INFO ]  Processing system.836.txt.
2020-06-19 22:30:09,601 [MainThread  ] [INFO ]  Processing system.188.txt.
2020-06-19 22:30:09,602 [MainThread  ] [INFO ]  Processing system.639.txt.
2020-06-19 22:30:09,603 [MainThread  ] [INFO ]  Processing system.163.txt.
2020-06-19 22:30:09,604 [MainThread  ] [INFO ]  Processing system.605.txt.
2020-06-19 22:30:09,605 [MainThread  ] [INFO ]  Processing system.611.txt.
2020-06-19 22:30:09,606 [MainThread  ] [INFO ]  Processing system.177.txt.
2020-06-19 22:30:09,607 [MainThread  ] [INFO ]  Processing system.956.txt.
2020-06-19 22:30:09,608 [MainThread  ] [INFO ]  Processing system.942.txt.
2020-06-19 22:30:09,609 [M

2020-06-19 22:30:09,793 [MainThread  ] [INFO ]  Processing system.879.txt.
2020-06-19 22:30:09,794 [MainThread  ] [INFO ]  Processing system.9.txt.
2020-06-19 22:30:09,795 [MainThread  ] [INFO ]  Processing system.448.txt.
2020-06-19 22:30:09,796 [MainThread  ] [INFO ]  Processing system.312.txt.
2020-06-19 22:30:09,797 [MainThread  ] [INFO ]  Processing system.474.txt.
2020-06-19 22:30:09,798 [MainThread  ] [INFO ]  Processing system.35.txt.
2020-06-19 22:30:09,799 [MainThread  ] [INFO ]  Processing system.21.txt.
2020-06-19 22:30:09,799 [MainThread  ] [INFO ]  Processing system.460.txt.
2020-06-19 22:30:09,800 [MainThread  ] [INFO ]  Processing system.306.txt.
2020-06-19 22:30:09,801 [MainThread  ] [INFO ]  Processing system.1012.txt.
2020-06-19 22:30:09,802 [MainThread  ] [INFO ]  Processing system.1006.txt.
2020-06-19 22:30:09,803 [MainThread  ] [INFO ]  Processing system.1010.txt.
2020-06-19 22:30:09,804 [MainThread  ] [INFO ]  Processing system.1004.txt.
2020-06-19 22:30:09,805 [

2020-06-19 22:30:09,892 [MainThread  ] [INFO ]  Processing system.923.txt.
2020-06-19 22:30:09,893 [MainThread  ] [INFO ]  Processing system.102.txt.
2020-06-19 22:30:09,894 [MainThread  ] [INFO ]  Processing system.664.txt.
2020-06-19 22:30:09,895 [MainThread  ] [INFO ]  Processing system.670.txt.
2020-06-19 22:30:09,896 [MainThread  ] [INFO ]  Processing system.116.txt.
2020-06-19 22:30:09,897 [MainThread  ] [INFO ]  Processing system.658.txt.
2020-06-19 22:30:09,898 [MainThread  ] [INFO ]  Processing system.880.txt.
2020-06-19 22:30:09,899 [MainThread  ] [INFO ]  Processing system.894.txt.
2020-06-19 22:30:09,899 [MainThread  ] [INFO ]  Processing system.843.txt.
2020-06-19 22:30:09,900 [MainThread  ] [INFO ]  Processing system.857.txt.
2020-06-19 22:30:09,901 [MainThread  ] [INFO ]  Processing system.27.txt.
2020-06-19 22:30:09,902 [MainThread  ] [INFO ]  Processing system.466.txt.
2020-06-19 22:30:09,903 [MainThread  ] [INFO ]  Processing system.300.txt.
2020-06-19 22:30:09,904 [M

...

2020-06-19 22:30:11,491 [MainThread  ] [INFO ]  Processing model.A.490.txt.
2020-06-19 22:30:11,492 [MainThread  ] [INFO ]  Processing model.A.484.txt.
2020-06-19 22:30:11,492 [MainThread  ] [INFO ]  Processing model.A.269.txt.
2020-06-19 22:30:11,493 [MainThread  ] [INFO ]  Processing model.A.241.txt.
2020-06-19 22:30:11,494 [MainThread  ] [INFO ]  Processing model.A.527.txt.
2020-06-19 22:30:11,495 [MainThread  ] [INFO ]  Processing model.A.533.txt.
2020-06-19 22:30:11,496 [MainThread  ] [INFO ]  Processing model.A.255.txt.
2020-06-19 22:30:11,497 [MainThread  ] [INFO ]  Processing model.A.11.txt.
2020-06-19 22:30:11,498 [MainThread  ] [INFO ]  Processing model.A.282.txt.
2020-06-19 22:30:11,499 [MainThread  ] [INFO ]  Processing model.A.296.txt.
2020-06-19 22:30:11,500 [MainThread  ] [INFO ]  Processing model.A.39.txt.
2020-06-19 22:30:11,501 [MainThread  ] [INFO ]  Processing model.A.719.txt.
2020-06-19 22:30:11,501 [MainThread  ] [INFO ]  Processing model.A.725.txt.
2020-06-19 22:

2020-06-19 22:30:11,587 [MainThread  ] [INFO ]  Processing model.A.871.txt.
2020-06-19 22:30:11,587 [MainThread  ] [INFO ]  Processing model.A.865.txt.
2020-06-19 22:30:11,588 [MainThread  ] [INFO ]  Processing model.A.695.txt.
2020-06-19 22:30:11,589 [MainThread  ] [INFO ]  Processing model.A.681.txt.
2020-06-19 22:30:11,590 [MainThread  ] [INFO ]  Processing model.A.859.txt.
2020-06-19 22:30:11,591 [MainThread  ] [INFO ]  Processing model.A.118.txt.
2020-06-19 22:30:11,592 [MainThread  ] [INFO ]  Processing model.A.656.txt.
2020-06-19 22:30:11,593 [MainThread  ] [INFO ]  Processing model.A.130.txt.
2020-06-19 22:30:11,594 [MainThread  ] [INFO ]  Processing model.A.124.txt.
2020-06-19 22:30:11,595 [MainThread  ] [INFO ]  Processing model.A.642.txt.
2020-06-19 22:30:11,596 [MainThread  ] [INFO ]  Processing model.A.125.txt.
2020-06-19 22:30:11,597 [MainThread  ] [INFO ]  Processing model.A.643.txt.
2020-06-19 22:30:11,598 [MainThread  ] [INFO ]  Processing model.A.657.txt.
2020-06-19 2

2020-06-19 22:30:11,683 [MainThread  ] [INFO ]  Processing model.A.779.txt.
2020-06-19 22:30:11,684 [MainThread  ] [INFO ]  Processing model.A.989.txt.
2020-06-19 22:30:11,684 [MainThread  ] [INFO ]  Processing model.A.751.txt.
2020-06-19 22:30:11,685 [MainThread  ] [INFO ]  Processing model.A.745.txt.
2020-06-19 22:30:11,686 [MainThread  ] [INFO ]  Processing model.A.976.txt.
2020-06-19 22:30:11,687 [MainThread  ] [INFO ]  Processing model.A.962.txt.
2020-06-19 22:30:11,688 [MainThread  ] [INFO ]  Processing model.A.792.txt.
2020-06-19 22:30:11,689 [MainThread  ] [INFO ]  Processing model.A.786.txt.
2020-06-19 22:30:11,690 [MainThread  ] [INFO ]  Processing model.A.948.txt.
2020-06-19 22:30:11,690 [MainThread  ] [INFO ]  Processing model.A.790.txt.
2020-06-19 22:30:11,691 [MainThread  ] [INFO ]  Processing model.A.784.txt.
2020-06-19 22:30:11,692 [MainThread  ] [INFO ]  Processing model.A.974.txt.
2020-06-19 22:30:11,693 [MainThread  ] [INFO ]  Processing model.A.960.txt.
2020-06-19 2

2020-06-19 22:30:11,779 [MainThread  ] [INFO ]  Processing model.A.187.txt.
2020-06-19 22:30:11,780 [MainThread  ] [INFO ]  Processing model.A.839.txt.
2020-06-19 22:30:11,781 [MainThread  ] [INFO ]  Processing model.A.193.txt.
2020-06-19 22:30:11,782 [MainThread  ] [INFO ]  Processing model.A.178.txt.
2020-06-19 22:30:11,782 [MainThread  ] [INFO ]  Processing model.A.144.txt.
2020-06-19 22:30:11,783 [MainThread  ] [INFO ]  Processing model.A.622.txt.
2020-06-19 22:30:11,784 [MainThread  ] [INFO ]  Processing model.A.636.txt.
2020-06-19 22:30:11,785 [MainThread  ] [INFO ]  Processing model.A.150.txt.
2020-06-19 22:30:11,786 [MainThread  ] [INFO ]  Processing model.A.637.txt.
2020-06-19 22:30:11,787 [MainThread  ] [INFO ]  Processing model.A.151.txt.
2020-06-19 22:30:11,787 [MainThread  ] [INFO ]  Processing model.A.145.txt.
2020-06-19 22:30:11,788 [MainThread  ] [INFO ]  Processing model.A.623.txt.
2020-06-19 22:30:11,789 [MainThread  ] [INFO ]  Processing model.A.179.txt.
2020-06-19 2

2020-06-19 22:30:11,875 [MainThread  ] [INFO ]  Processing model.A.60.txt.
2020-06-19 22:30:11,876 [MainThread  ] [INFO ]  Processing model.A.74.txt.
2020-06-19 22:30:11,877 [MainThread  ] [INFO ]  Processing model.A.754.txt.
2020-06-19 22:30:11,878 [MainThread  ] [INFO ]  Processing model.A.998.txt.
2020-06-19 22:30:11,879 [MainThread  ] [INFO ]  Processing model.A.740.txt.
2020-06-19 22:30:11,880 [MainThread  ] [INFO ]  Processing model.A.768.txt.
2020-06-19 22:30:11,881 [MainThread  ] [INFO ]  Processing model.A.797.txt.
2020-06-19 22:30:11,882 [MainThread  ] [INFO ]  Processing model.A.783.txt.
2020-06-19 22:30:11,883 [MainThread  ] [INFO ]  Processing model.A.973.txt.
2020-06-19 22:30:11,884 [MainThread  ] [INFO ]  Processing model.A.967.txt.
2020-06-19 22:30:11,885 [MainThread  ] [INFO ]  Saved processed files to /var/folders/0c/zfyk8fc527xfppxw_pkwm0br0000gn/T/tmpys1wssh7/model.
2020-06-19 22:30:13,157 [MainThread  ] [INFO ]  Written ROUGE configuration to /var/folders/0c/zfyk8f

In [47]:
def get_rouge_12l(output_dict):
    scores = {}
    scores['rouge_1'] = {'p': output_dict['rouge_1_precision'], 'r': output_dict['rouge_1_recall'], 'f': output_dict['rouge_1_f_score']}
    scores['rouge_2'] = {'p': output_dict['rouge_2_precision'], 'r': output_dict['rouge_2_recall'], 'f': output_dict['rouge_2_f_score']}
    scores['rouge_l'] = {'p': output_dict['rouge_l_precision'], 'r': output_dict['rouge_l_recall'], 'f': output_dict['rouge_l_f_score']}
    return scores

In [48]:
lex_rank_scores = get_rouge_12l(lex_rank_rouge[1])
edmunson_scores = get_rouge_12l(edmunson_rouge[1])
leadcm_scores = get_rouge_12l(leadcm_rouge[1])


In [58]:
def write_to_txt(scores, output_path):
    with open(output_path, 'w+') as fn:
        fn.write(str(scores))

In [59]:
OUTPUT = '/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/scores'
write_to_txt(lex_rank_rouge[0], f'{OUTPUT}/lex_rank_rouge.txt')
write_to_txt(edmunson_rouge[0], f'{OUTPUT}/edmunson_rouge.txt')
write_to_txt(leadcm_rouge[0], f'{OUTPUT}/leadcm_rouge.txt')

In [57]:
lex_rank_rouge[0]

'---------------------------------------------\n1 ROUGE-1 Average_R: 0.19185 (95%-conf.int. 0.17884 - 0.20524)\n1 ROUGE-1 Average_P: 0.36498 (95%-conf.int. 0.34858 - 0.38018)\n1 ROUGE-1 Average_F: 0.21596 (95%-conf.int. 0.20326 - 0.22952)\n---------------------------------------------\n1 ROUGE-2 Average_R: 0.08988 (95%-conf.int. 0.07705 - 0.10251)\n1 ROUGE-2 Average_P: 0.15189 (95%-conf.int. 0.13579 - 0.16812)\n1 ROUGE-2 Average_F: 0.09857 (95%-conf.int. 0.08549 - 0.11165)\n---------------------------------------------\n1 ROUGE-3 Average_R: 0.07044 (95%-conf.int. 0.05829 - 0.08287)\n1 ROUGE-3 Average_P: 0.11100 (95%-conf.int. 0.09516 - 0.12661)\n1 ROUGE-3 Average_F: 0.07539 (95%-conf.int. 0.06297 - 0.08782)\n---------------------------------------------\n1 ROUGE-4 Average_R: 0.06143 (95%-conf.int. 0.04965 - 0.07300)\n1 ROUGE-4 Average_P: 0.09419 (95%-conf.int. 0.07937 - 0.10948)\n1 ROUGE-4 Average_F: 0.06468 (95%-conf.int. 0.05268 - 0.07666)\n-------------------------------------------

# Write the Rouge Score of Extractive Summarization on the Test Set to the Dataframe and export to csv

In [62]:
df = pd.read_csv('/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/extractive_summarizations/extractive_summarizations.csv')

In [64]:
df = df.drop('Unnamed: 0', 1)

In [65]:
df = df.drop('readme', 1)

In [67]:
df.to_csv('/Users/vincenthuang/Development/Summer-2020/readme_summarizer/data/extractive_summarizations/extractive_summarizations_without_readme.csv')